In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

import numpy as np

import matplotlib.pyplot as plt

from rec.models.resnet_vae import BidirectionalResNetVAE

In [2]:
brn_vae = BidirectionalResNetVAE(num_res_blocks=24)

In [3]:
test_in = tf.ones((10, 32, 32, 3))

brn_vae(test_in)

<tf.Tensor: id=10609, shape=(10, 32, 32, 3), dtype=float32, numpy=
array([[[[0.49180222, 0.5335027 , 0.4972045 ],
         [0.49771896, 0.5278611 , 0.52565587],
         [0.5106337 , 0.5461607 , 0.45746794],
         ...,
         [0.5383668 , 0.5666442 , 0.5216496 ],
         [0.5106337 , 0.5461607 , 0.45746794],
         [0.51449233, 0.5193336 , 0.51242405]],

        [[0.46631405, 0.54408115, 0.5033677 ],
         [0.44988054, 0.4181423 , 0.5177617 ],
         [0.47988296, 0.5033522 , 0.5273672 ],
         ...,
         [0.41065636, 0.4415706 , 0.47916123],
         [0.47988296, 0.5033522 , 0.5273672 ],
         [0.4355563 , 0.47428232, 0.47509503]],

        [[0.512375  , 0.53458256, 0.4732344 ],
         [0.46922898, 0.54499024, 0.49188554],
         [0.5171748 , 0.54714257, 0.45055577],
         ...,
         [0.5308933 , 0.56725067, 0.4494638 ],
         [0.5171748 , 0.54714257, 0.45055577],
         [0.52387315, 0.50085735, 0.46467337]],

        ...,

        [[0.44862062, 0.5

In [10]:
kls = brn_vae.kl_divergence(empirical=True)

In [12]:
[tf.reduce_any(tf.math.is_inf(kl)) for kl in kls]

[<tf.Tensor: id=13510, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13513, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13516, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13519, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13522, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13525, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13528, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13531, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13534, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13537, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13540, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13543, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13546, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13549, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13552, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13555, shape=(), dtype=bool, numpy=False>,
 <tf.Tensor: id=13558, shape=(), dtype=bool, numpy=False